In [73]:
from huggingface_hub import HfApi, DatasetSearchArguments, DatasetFilter
import datasets
from datasets import load_dataset
from datasets import list_datasets
from datasets import get_dataset_config_names

api = HfApi()

In [16]:
# cnn_dataset = load_dataset("cnn_dailymail", "3.0.0")
# cnn_dataset.save_to_disk("datasets/cnn_dailymail")

In [17]:

# List all datasets
api.list_datasets()

<generator object HfApi.list_datasets at 0x000002A96C114C80>

In [18]:

# Get all valid search arguments
args = DatasetSearchArguments()


def get_tag_value(tag_name, dataset):
    # Initialize an empty list to store the tag values
    tag_values = []

    # Loop through the tags of each item
    for tag in dataset.tags:
        # Split the tag into its name and value
        tag_parts = tag.split(":")

        # Check if the tag name matches the one you are interested in
        if tag_parts[0] == tag_name:
            # Append the value part of the tag to the list
            tag_values.append(tag_parts[1])

    return tag_values


KeyboardInterrupt: 

In [ ]:
#Create the filters with for the database request
filt = DatasetFilter(task_categories=args.task_categories.summarization, language='en', multilinguality="monolingual")
my_list = api.list_datasets(filter=filt)

In [ ]:
# Initialize sets for tags and classifications
my_tags = set()
my_classifications = set()

# Loop through your list
for item in my_list:
    print(item)
    for tag in item.tags:
        # If the tag contains a colon, it has sub-information
        if ":" in tag:
            my_tags.add(tag.split(":")[0])
        # Otherwise, it is a classification
        else:
            my_classifications.add(tag)

print("Tags:", my_tags)
print("Classifications:", my_classifications)


In [ ]:
my_items = []
for item in my_list:
    for tag in item.tags:
        tagItem = tag.split(":")
        if tagItem[0].lower() == "license":
            print(f"{item.id} license: {tagItem[1]}")

In [ ]:
print(datasets.get_dataset_config_info("cnn_dailymail", "3.0.0", download_mode='reuse_cache_if_exists'))
print(datasets.get_dataset_config_info("cnn_dailymail", "2.0.0", download_mode='reuse_cache_if_exists'))
print(datasets.get_dataset_config_info("cnn_dailymail", "1.0.0", download_mode='reuse_cache_if_exists'))

In [38]:
filt = DatasetFilter(task_categories=args.task_categories.summarization, language='en', multilinguality="monolingual")
my_list = api.list_datasets(filter=filt)

# Look out features on different datasets.

In [27]:
features_configurations = set()
for item in my_list:
    try:
        config_name = datasets.get_dataset_config_names(item.id, download_mode='reuse_dataset_if_exists')
        if config_name:
            for config in config_name:
                configs = datasets.get_dataset_config_info(item.id, config, download_mode='reuse_dataset_if_exists')
                if configs.features is not None:
                    features_configurations.add(frozenset(configs.features.keys()))
        else:
            configs = datasets.get_dataset_config_info(item.id, download_mode='reuse_dataset_if_exists')
            if configs.features is not None:
                features_configurations.add(frozenset(configs.features.keys()))
    except Exception as e:
        print(f"{item.id} skipped unknown reason: {e}")

GroNLP/ik-nlp-22_slp skipped unknown reason: Python int too large to convert to C long
Hellisotherpeople/DebateSum skipped unknown reason: 'NoneType' object has no attribute 'keys'
ccdv/cnn_dailymail skipped unknown reason: The split names could not be parsed from the dataset config.
DebateLabKIT/aaac skipped unknown reason: 'NoneType' object has no attribute 'keys'
lukesjordan/worldbank-project-documents skipped unknown reason: 'NoneType' object has no attribute 'keys'
stevhliu/demo skipped unknown reason: 'NoneType' object has no attribute 'keys'
shanya/crd3 skipped unknown reason: The split names could not be parsed from the dataset config.
alexfabbri/answersumm skipped unknown reason: 'NoneType' object has no attribute 'keys'
knkarthick/dialogsum skipped unknown reason: 'NoneType' object has no attribute 'keys'
knkarthick/AMI skipped unknown reason: Couldn't find a dataset script at C:\Users\PC\PycharmProjects\TLDR\scripts\knkarthick\AMI\AMI.py or any data file in the same director

Resolving data files:   0%|          | 0/17 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/17 [00:00<?, ?it/s]

neil-code/dialogsum-test skipped unknown reason: 'NoneType' object has no attribute 'keys'


In [28]:
features_configurations

{frozenset({'abstract', 'description'}),
 frozenset({'compression',
            'compression_bin',
            'coverage',
            'coverage_bin',
            'date',
            'density',
            'density_bin',
            'summary',
            'text',
            'title',
            'url'}),
 frozenset({'doc_id', 'document_sections', 'question_summary_pairs'}),
 frozenset({'abstract', 'article', 'section_names'}),
 frozenset({'directions',
            'id',
            'ingredients',
            'link',
            'ner',
            'source',
            'title'}),
 frozenset({'category', 'content', 'headline'}),
 frozenset({'CHQ', 'File', 'Summary'}),
 frozenset({'doc_id',
            'document_sections',
            'question_summary_pairs',
            'summary_paragraph_index'}),
 frozenset({'id', 'sources', 'summary/long', 'summary/short', 'summary/tiny'}),
 frozenset({'gem_id',
            'gem_parent_id',
            'meaning_representation',
            'reference

# Look out by description for news articles and licenses

In [30]:
news_keywords = ["news", "headline", "journalism", "report", "media", "press"]

news_related_datasets = []

for item in my_list:
    try:
        config_names = datasets.get_dataset_config_names(item.id, download_mode='reuse_cache_if_exists')
        if config_names:
            for config_name in config_names:
                config = datasets.get_dataset_config_info(item.id, config_name, download_mode='reuse_cache_if_exists')
                if config.features is not None:
                    if any(keyword.lower() in config.description.lower() for keyword in news_keywords):
                        if item.id not in news_related_datasets:
                            news_related_datasets[item.id] = []
                        news_related_datasets[item.id].append(config)
        else:
            config = datasets.get_dataset_config_info(item.id, download_mode='reuse_cache_if_exists')
            if config.features is not None:
                if any(keyword.lower() in config.description.lower() for keyword in news_keywords):
                    if item.id not in news_related_datasets:
                        news_related_datasets[item.id] = []
                    news_related_datasets[item.id].append(config)
    except Exception:
        continue

Resolving data files:   0%|          | 0/17 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/17 [00:00<?, ?it/s]

In [ ]:
news_keywords = ["news", "headline", "journalism", "report", "media", "press"]
filt = DatasetFilter(task_categories=args.task_categories.summarization, language='en', multilinguality="monolingual")
my_list = api.list_datasets(filter=filt)

# Initialize as a dictionary
news_related_datasets = {}

for item in my_list:
    try:
        config_names = datasets.get_dataset_config_names(item.id, download_mode='reuse_cache_if_exists')
        config_names = config_names if config_names else [None]

        for config_name in config_names:
            config = datasets.get_dataset_config_info(item.id, config_name, download_mode='reuse_cache_if_exists')
            if config and config.features and config.description:
                if any(keyword.lower() in config.description.lower() for keyword in news_keywords):
                    if item.id not in news_related_datasets:
                        news_related_datasets[item.id] = {'configs': [], 'license': None}
                    news_related_datasets[item.id]['configs'].append(config)

        if item.id in news_related_datasets and news_related_datasets[item.id]['configs']:
            news_related_datasets[item.id]['license'] = get_tag_value('license', item)
    except Exception as e:  # Consider catching specific exceptions
        print(f"Exception encountered for item id {item.id if item and item.id else 'Unknown'}: {e}")

# Check if news_related_datasets is empty
if not any(val['configs'] for val in news_related_datasets.values()):
    print("No news-related datasets found.")
else:
    print(f"Found {len(news_related_datasets)} news-related datasets.")


In [72]:
news_keywords = ["news", "headline", "journalism", "report", "media", "press"]
filt = DatasetFilter(task_categories=args.task_categories.summarization, language='en', multilinguality="monolingual")
my_list = api.list_datasets(filter=filt)
# Initialize as a dictionary
news_related_datasets = {}

for item in my_list:
    try:
        config_names = datasets.get_dataset_config_names(item.id, download_mode='reuse_cache_if_exists')
        if item.id not in news_related_datasets:
            news_related_datasets[item.id] = {'configs': [], 'license': None}

        if config_names:
            for config_name in config_names:
                config = datasets.get_dataset_config_info(item.id, config_name, download_mode='reuse_cache_if_exists')
                if config and config.features and config.description:
                    if any(keyword.lower() in config.description.lower() for keyword in news_keywords):
                        news_related_datasets[item.id]['configs'].append(config)
        else:
            config = datasets.get_dataset_config_info(item.id, download_mode='reuse_cache_if_exists')
            if config and config.features and config.description:
                if any(keyword.lower() in config.description.lower() for keyword in news_keywords):
                    news_related_datasets[item.id]['configs'].append(config)

        if news_related_datasets[item.id]['configs']:
            news_related_datasets[item.id]['license'] = get_tag_value('license', item)
    except Exception as e:
        print(f"Exception encountered for item id {item.id if item and item.id else 'Unknown'}: {e}")

# Check if news_related_datasets is empty
if not any(val['configs'] for val in news_related_datasets.values()):
    print("No news-related datasets found.")
else:
    print(f"Found {len(news_related_datasets)} news-related datasets.")

Exception encountered for item id GroNLP/ik-nlp-22_slp: Python int too large to convert to C long
Exception encountered for item id ccdv/cnn_dailymail: The split names could not be parsed from the dataset config.


Exception encountered for item id shanya/crd3: The split names could not be parsed from the dataset config.
Exception encountered for item id knkarthick/AMI: Couldn't find a dataset script at C:\Users\PC\PycharmProjects\TLDR\scripts\knkarthick\AMI\AMI.py or any data file in the same directory. Couldn't find 'knkarthick/AMI' on the Hugging Face Hub either: FileNotFoundError: Dataset 'knkarthick/AMI' doesn't exist on the Hub. If the repo is private or gated, make sure to log in with `huggingface-cli login`.
Exception encountered for item id RUCAIBox/Summarization: Couldn't find a dataset script at C:\Users\PC\PycharmProjects\TLDR\scripts\RUCAIBox\Summarization\Summarization.py or any data file in the same directory. Couldn't find 'RUCAIBox/Summarization' on the Hugging Face Hub either: FileNotFoundError: No (supported) data files or dataset script found in RUCAIBox/Summarization. 
Exception encountered for item id NavidVafaei/rottentomato01: array() takes at least 1 positional argument (

Resolving data files:   0%|          | 0/17 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/17 [00:00<?, ?it/s]

Found 102 news-related datasets.


In [ ]:
news_related_datasets

In [71]:
for dataset_id, configs in news_related_datasets.items():
    for config in configs:
        if hasattr(config, 'features'):
            if len(config.features) < 4:
                print(f"{dataset_id}: {config.features.keys()}, size: {config.dataset_size}")
        break

In [35]:
# Output the IDs of news-related datasets along with their configurations
for dataset_id, config_names in news_related_datasets:
    if config_names:
        print(f"News-related dataset: {dataset_id}, Configurations: {', '.join(config_names)}")
    else:
        print(f"News-related dataset: {dataset_id}, No specific configuration.")

# Getting tags from datasets

In [ ]:
from datasets import load_dataset

# List of acceptable licenses
acceptable_licenses = ['MIT', 'Apache-2.0', 'CC0', 'CC BY', 'CC BY-SA']

# Loop through each dataset to check license and configurations
for dataset_id in my_list:
    # You can get information about the dataset without downloading it using `load_dataset` with `download_mode='reuse_cache_if_exists'`
    dataset = load_dataset(dataset_id.id, download_mode='reuse_cache_if_exists', split='train[:0]')  # Split set to 'train[:0]' to download meta-data only
    dataset_info = dataset.info

    # Check license
    license_type = dataset_info.license  # Update this line based on how license information is stored in `dataset_info`
    if license_type not in acceptable_licenses:
        print(f"Skipping {dataset_id.id} due to incompatible license: {license_type}")
        continue

    # Check configurations
    config_names = dataset_info.config_name  # Replace this with the appropriate way to get configuration names
    print(f"Available configurations for {dataset_id.id}: {config_names}")

    # Decide whether to download the dataset
    # user_input = input(f"Do you want to download {dataset_id.id}? (y/n): ")
    # if user_input.lower() == 'y':
    #     # Your download code here. For example:
    #     dataset = load_dataset(dataset_id, split='train')  # This will download the dataset


In [ ]:
from IPython.display import display
from ipywidgets import Dropdown
from datasets import load_dataset

# Your existing acceptable_licenses and my_list
acceptable_licenses = ['MIT', 'Apache-2.0', 'CC0', 'CC BY', 'CC BY-SA']

config_dropdowns = []

# Loop through each dataset to check license and configurations
for dataset_id in my_list:
    dataset = load_dataset(dataset_id.id, download_mode='reuse_cache_if_exists', split='train[:0]')
    dataset_info = dataset.info

    # Check license
    license_type = dataset_info.license
    if license_type not in acceptable_licenses:
        print(f"Skipping {dataset_id.id} due to incompatible license: {license_type}")
        continue

    # Check configurations
    config_names = dataset_info.builder_configs  # The attribute might be different; replace accordingly
    config_names = [config.name for config in config_names]

    if config_names:
        print(f"Available configurations for {dataset_id.id}: {config_names}")

        dropdown = Dropdown(options=config_names, description=dataset_id.id)
        config_dropdowns.append(dropdown)
        display(dropdown)

# Later, you can collect the user selections:
selected_configs = [(dropdown.description, dropdown.value) for dropdown in config_dropdowns]

# Now you have a list of tuples: [('dataset_id1', 'selected_config1'), ('dataset_id2', 'selected_config2'), ...]
# You can proceed to download these selected configurations.


In [ ]:
import tkinter as tk
from tkinter import ttk
from datasets import load_dataset

# Function to handle the "Download" button click
def on_download_click(dataset_id, config_value, license_type):
    print(f"Downloading {dataset_id} with config {config_value} and license {license_type}")
    # Add your download code here

# Function to handle the "Skip" button click
def on_skip_click(dataset_id):
    print(f"Skipping {dataset_id}")

# Initialize Tkinter window
root = tk.Tk()
root.title('Dataset Downloader')
filt = DatasetFilter(task_categories=args.task_categories.summarization, language='en', multilinguality="monolingual")
my_list = api.list_datasets(filter=filt)

# Loop through each dataset to create UI elements
for dataset in my_list:
    dataset_id = dataset.id
    config_options = datasets.get_dataset_config_names(dataset_id)
    license_type = dataset.tags[7]

    frame = ttk.Frame(root, padding="10")
    frame.grid(row=0, column=0, sticky=(tk.W, tk.E))

    ttk.Label(frame, text=f"Dataset ID: {dataset_id}").grid(row=0, column=0, sticky=tk.W)
    ttk.Label(frame, text=f"License: {license_type}").grid(row=1, column=0, sticky=tk.W)

    config_value = tk.StringVar(value=config_options[0])
    config_combo = ttk.Combobox(frame, textvariable=config_value)
    config_combo['values'] = config_options
    config_combo.grid(row=2, column=0, sticky=tk.W)

    ttk.Button(frame, text="Download", command=lambda dataset_id=dataset_id, config_value=config_value, license_type=license_type: on_download_click(dataset_id, config_value.get(), license_type)).grid(row=3, column=0)
    ttk.Button(frame, text="Skip", command=lambda dataset_id=dataset_id: on_skip_click(dataset_id)).grid(row=3, column=1)

root.mainloop()


## Remove from dataset in api

In [ ]:
import os
import shutil

def remove_dataset_from_cache(dataset_name, cache_dir=None):
    if cache_dir is None:
        # Replace this with the actual default cache directory path if it's different
        cache_dir = os.path.join(os.path.expanduser("~"), ".cache", "huggingface", "datasets")

    dataset_path = os.path.join(cache_dir, dataset_name)

    if os.path.exists(dataset_path):
        # Remove the dataset directory
        shutil.rmtree(dataset_path)
        print(f"Successfully removed {dataset_name} from cache.")
    else:
        print(f"Dataset {dataset_name} not found in cache.")

# Example usage:
remove_dataset_from_cache('squad')

In [ ]:
configs

In [ ]:
# Initialize an empty set to store unique keys
unique_keys = set()

# Loop through each DatasetInfo object to collect keys
for dataset_info in my_list:
    unique_keys.add(dataset_info.id)
